## 専門知識労働者

## extra jupyterノートブック -  4.5日 - ファイスのクロマを切り替えてください！

FAISSはFacebook AIの類似性検索です

In [ ]:
# 輸入

import os
import glob
from dotenv import load_dotenv
import gradio as gr

In [ ]:
# FAISS-CPUをインストールしてください！
# Macユーザー - 最近のバージョンのmacosを持っていない場合、これは失敗する可能性があります
# その場合、このラボをスキップすることをお勧めします。FAISSは必須ではありません！ （または、必要に応じてmacosをアップグレードします。）

!pip install faiss-cpu

In [ ]:
# Langchainの輸入

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
# langchain_chromaインポートクロマから
from langchain.vectorstores import FAISS
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [ ]:
# 価格は当社の要因であるため、低コストモデルを使用します

MODEL = "gpt-4o-mini"
db_name = "vector_db"

In [ ]:
# .envというファイルに環境変数をロードします

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

In [ ]:
# Langchainのローダーを使用してドキュメントを読んでください
# 知識ベースのすべてのサブフォルダーにすべてを取ります

folders = glob.glob("knowledge-base/*")

# 一部のユーザーに必要なこの修正については、コースの学生であるCGとJon Rに感謝します
text_loader_kwargs = {'encoding': 'utf-8'}
# それがうまくいかない場合、一部のWindowsユーザーは代わりに次の行を除外する必要があるかもしれません
# text_loader_kwargs = {'autodetect_encoding'：true}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [ ]:
len(chunks)

In [ ]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

## 埋め込みのサイドノート、および「自動エンコードLLMS」

テキストの各塊を、埋め込みとして知られるテキストの意味を表すベクトルにマッピングします。

Openaiはこれを行うためのモデルを提供します。これをLangchainコードでAPIを呼び出すことで使用します。

このモデルは、完全な入力が与えられた出力を生成する「自動エンコードLLM」の例です。
「自動回帰LLM」として知られている今日説明した他のすべてのLLMとは異なり、過去のコンテキストのみに基づいて将来のトークンを生成します。

自動エンコードLLMの別の例は、GoogleのBertです。埋め込みに加えて、自動エンコードLLMが分類に使用されることがよくあります。

###サイドノート

第8週には、RAGおよびベクトルの埋め込みに戻り、オープンソースベクトルエンコーダを使用して、データがコンピューターを離れることはありません。これは、エンタープライズシステムとデータが内部を維持する必要がある場合に重要な考慮事項です。

In [ ]:
# データの塊を、各チャンクと埋め込むベクトルを関連付けるベクトルストアに入れます
# ChromaはSQLLITEに基づいた人気のオープンソースベクトルデータベースです

embeddings = OpenAIEmbeddings()

# VectorStoreを作成します

# 前に
# vectorStore = chroma.from_documents（documents = chunks、embedding = embeddings、fersting_directory = db_name）

# 後
vectorstore = FAISS.from_documents(chunks, embedding=embeddings)

total_vectors = vectorstore.index.ntotal
dimensions = vectorstore.index.d

print(f"There are {total_vectors} vectors with {dimensions:,} dimensions in the vector store")

In [ ]:
# プレウォーク
vectors = []
documents = []
doc_types = []
colors = []
color_map = {'products':'blue', 'employees':'green', 'contracts':'red', 'company':'orange'}

for i in range(total_vectors):
    vectors.append(vectorstore.index.reconstruct(i))
    doc_id = vectorstore.index_to_docstore_id[i]
    document = vectorstore.docstore.search(doc_id)
    documents.append(document.page_content)
    doc_type = document.metadata['doc_type']
    doc_types.append(doc_type)
    colors.append(color_map[doc_type])
    
vectors = np.array(vectors)

## ベクトルストアの視覚化

ドキュメントとその埋め込みベクトルを見て、何が起こっているのかを確認しましょう。

（サイドノートとして、私たちが実際にここで見ているのは、FAISSから取得されたOpenAiembedingsによって生成されたベクトルの分布です。したがって、それらが「FaissまたはChromaから」であるかどうかを驚かせることはありません。）

In [ ]:
# 私たち人間は、2Dで物事を視覚化する方が簡単だと感じています！
# T-SNEを使用して、ベクターの寸法を2Dに減らします
# （t分配された確率的隣接埋め込み）

tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# 2D散布図を作成します
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='2D FAISS Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [ ]:
# 3Dを試してみましょう！

tsne = TSNE(n_components=3, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# 3D散布図を作成します
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D FAISS Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=900,
    height=700,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

## Langchainを使用してすべてをまとめる時間です

In [ ]:
# Openaiとの新しいチャットを作成します
llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# チャットの会話メモリを設定します
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# レトリーバーは、ぼろきれに使用されるベクターストアを抽象化しています
retriever = vectorstore.as_retriever()

# それをまとめる：GPT 3.5 LLM、ベクトルストア、メモリで会話チェーンをセットアップします
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [ ]:
query = "Can you describe Insurellm in a few sentences"
result = conversation_chain.invoke({"question":query})
print(result["answer"])

In [ ]:
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

## チャットインターフェイスを使用して、これをグラデーションで紹介します - 

LLMとのチャットをプロトタイプする迅速かつ簡単な方法

In [ ]:
# それを関数で包みます

def chat(message, history):
    result = conversation_chain.invoke({"question": message})
    return result["answer"]

In [ ]:
# そしてグラデーションで：

view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)